In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)
from utils.downloads import _prepare_cache

# População a menos de 1km de parques urbanos e lineares

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende do arquivo resultante do notebook 'pop_1km_parques_urb_line'

In [5]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'pop_1km_parques_urbanos_lineares.parquet'
    )
)

# Calcular a porcentagem da pop_recort

Já temos a quantidade de pessoas por distrito, mas o indicador é: % da população a menos de 1 km de parques urbanos e lineares, que é 

População total (do distrito) / População a 1km ou menos de parques (no distrito)

In [6]:
gdf_distritos['pct_pop_recort']= (
    gdf_distritos['pop_dist_recort']/gdf_distritos['total_pop']
)

In [7]:
gdf_distritos.sample(3)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
41,3550308,São Paulo,355030842,JARAGUA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,28.004258,211617,84799,"POLYGON ((324277.651 7404843.442, 324281.328 7...",160967.0,0.760652
65,3550308,São Paulo,355030866,REPUBLICA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,2.323756,60825,42899,"POLYGON ((332414.569 7394031.775, 332398.803 7...",60474.0,0.994229
35,3550308,São Paulo,355030836,ITAIM PAULISTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.217145,205295,78141,"POLYGON ((359382.741 7402158.394, 359386.115 7...",204209.0,0.994710


# Conferir

Novamente vamos checar vendo se os mínimos e máximos fazem sentido.

Aqui tem os distritos que não tem nenhuma pessoa a 1km ou menos de parques lineares e urbanos.

In [8]:
(
    gdf_distritos
    .loc[gdf_distritos['pct_pop_recort']==gdf_distritos['pct_pop_recort']
    .min()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
51,3550308,São Paulo,355030852,MARSILAC,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,207.501260,11451,6139,"POLYGON ((333016.008 7354600.919, 333056.044 7...",0.0,0.0
67,3550308,São Paulo,355030868,SACOMA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,14.610797,261436,113894,"POLYGON ((338605.158 7385601.06, 338598.678 73...",0.0,0.0
78,3550308,São Paulo,355030879,SOCORRO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.261579,38051,16549,"POLYGON ((326832.519 7378799.896, 326814.871 7...",0.0,0.0


Aqui é o distrito com a menor quantidade de pessoas a 1km ou menos de parques lineares e urbanos, menos os que estão zerados

In [9]:
distritos_sem0=gdf_distritos.loc[gdf_distritos['pct_pop_recort']!=0.0]
(
    distritos_sem0
    .loc[distritos_sem0['pct_pop_recort']==distritos_sem0['pct_pop_recort']
    .min()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
26,3550308,São Paulo,355030827,CURSINO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.060052,103171,46155,"POLYGON ((335822.146 7385759.521, 335833.043 7...",131.0,0.00127


Aqui vemos o distrito com a maior quantidade de pessoas a 1km ou menos dos parques.

In [10]:
(
    gdf_distritos
    .loc[gdf_distritos['pct_pop_recort']==gdf_distritos['pct_pop_recort']
    .max()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
64,3550308,São Paulo,355030865,RAPOSO TAVARES,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.164313,117738,49006,"POLYGON ((319022.919 7388900.499, 318990.285 7...",117738.0,1.0
93,3550308,São Paulo,355030894,VILA SONIA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,10.066693,123748,53826,"POLYGON ((323710.87 7387720.911, 323703.262 73...",123748.0,1.0


# Salvar

In [11]:
save_parquet_excel(
    gdf_distritos,
    'pct_pop_1km_parques_urbanos_lineares',
    assets_path,
    'parques_lineares_e_urbanos'
)